In [1]:
pip install mtcnn

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
    --------------------------------------- 0.0/2.3 MB 660.6 kB/s eta 0:00:04
    --------------------------------------- 0.0/2.3 MB 660.6 kB/s eta 0:00:04
    --------------------------------------- 0.0/2.3 MB 217.9 kB/s eta 0:00:11
   - -------------------------------------- 0.1/2.3 MB 297.7 kB/s eta 0:00:08
   - -------------------------------------- 0.1/2.3 MB 467.6 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/2.3 MB 481.4 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/2.3 MB 448.2 kB/s eta 0:00:05
   --- ------------------------------------ 0.2/2.3 MB 562.0 kB/s eta 0:00:04
   ---- ----------------------------------- 0.2/2.3 MB 656.0 kB/s eta 0:00:04
   ---- --------

In [3]:
# import tarfile
# import os

# # Path to the .tgz file on Google Drive
# tgz_file_path = '/content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/3. Skin Tone Prediction /lfw.tgz'

# # Directory where you want to extract the images
# extracted_folder_path = '/content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/3. Skin Tone Prediction /Images'

# # Create the output directory if it doesn't exist
# os.makedirs(extracted_folder_path, exist_ok=True)

# # Extract the images
# with tarfile.open(tgz_file_path, 'r:gz') as tar:
#     tar.extractall(path=extracted_folder_path)

# print("Extraction complete.")


In [ ]:
parent_directory = '/content/drive/MyDrive/Data Science/Data Science Projects/5. Final Project/3. Skin Tone Prediction /Images/lfw'

# List the subdirectories in the parent directory
subdirectories = [subdir for subdir in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, subdir))]

# Count the number of subdirectories
num_subdirectories = len(subdirectories)

print(f"Number of subdirectories in the directory: {num_subdirectories}")

Number of subdirectories in the directory: 5749


In [4]:
import os
import shutil

# Source directory containing subfolders (each representing a person)
source_directory = r'C:\Users\hp\OneDrive\Desktop\VScode\AI_PROJECT_SKIN_TONE\img_align_celeba'

# Destination directory to store combined images
destination_directory = r'C:\Users\hp\OneDrive\Desktop\VScode\AI_PROJECT_SKIN_TONE\images'

# Create the destination directory if it doesn't exist
os.makedirs(destination_directory, exist_ok=True)

# Initialize a counter for sequential numbering
counter = 1

# Get the existing files in the destination directory
existing_files = os.listdir(destination_directory)
existing_numbers = set()
for filename in existing_files:
    if filename.startswith("Image_") and filename.endswith(".jpg"):
        try:
            number = int(filename.split("_")[1].split(".")[0])
            existing_numbers.add(number)
        except ValueError:
            pass

# Loop through each person's folder
for person_folder in os.listdir(source_directory):
    person_folder_path = os.path.join(source_directory, person_folder)

    # Make sure we are dealing with a directory
    if os.path.isdir(person_folder_path):
        # Get the list of image files in the person's folder
        image_files = [filename for filename in os.listdir(person_folder_path) if filename.lower().endswith(('.jpg', '.png', '.jpeg'))]

        # Copy the first image to the destination folder and rename it with sequential number
        if image_files:
            first_image_path = os.path.join(person_folder_path, image_files[0])

            while counter in existing_numbers:
                counter += 1

            new_image_filename = f"Image_{counter:04d}.jpg"  # Format the new filename
            new_image_path = os.path.join(destination_directory, new_image_filename)
            shutil.copy(first_image_path, new_image_path)

            # Increment the counter for the next image
            counter += 1

print("Image extraction, copying, and renaming complete.")


Image extraction, copying, and renaming complete.


In [5]:
# List the files in the directory
files_in_directory = os.listdir(destination_directory)

# Count the number of image files
num_image_files = sum(1 for file in files_in_directory if file.lower().endswith(('.jpg', '.png', '.jpeg')))

print(f"Number of image files in the directory: {num_image_files}")


Number of image files in the directory: 0


In [6]:
import os
from mtcnn import MTCNN
import cv2

source_directory = r'C:\Users\hp\OneDrive\Desktop\VScode\AI_PROJECT_SKIN_TONE\img_align_celeba'
destination_directory = r'C:\Users\hp\OneDrive\Desktop\VScode\AI_PROJECT_SKIN_TONE\images'

os.makedirs(destination_directory, exist_ok=True)

mtcnn = MTCNN()

batch_size = 10  # Number of images to process in a batch

image_filenames = sorted([filename for filename in os.listdir(source_directory) if filename.lower().endswith(('.jpg', '.jpeg', '.png'))])

for batch_start in range(0, len(image_filenames), batch_size):
    batch_end = batch_start + batch_size
    batch_image_filenames = image_filenames[batch_start:batch_end]

    for image_filename in batch_image_filenames:
        image_path = os.path.join(source_directory, image_filename)
        image = cv2.imread(image_path)

        try:
            faces = mtcnn.detect_faces(image)
        except Exception as e:
            print(f"Error detecting faces in {image_filename}: {e}")
            continue  # Skip this image and move to the next

        if len(faces) > 0:
            largest_face = max(faces, key=lambda f: f['box'][2] * f['box'][3])
            x, y, w, h = largest_face['box']

            min_face_size = 50
            if w > min_face_size and h > min_face_size:
                face = image[y:y+h, x:x+w]

                extracted_face_filename = f'{os.path.splitext(image_filename)[0]}_face.jpg'
                extracted_face_path = os.path.join(destination_directory, extracted_face_filename)

                cv2.imwrite(extracted_face_path, face)
            else:
                print(f"Face in {image_filename} is too small and will be skipped.")
        else:
            print(f"No face detected in {image_filename}.")

    print(f"Processed images {batch_start+1} to {min(batch_end, len(image_filenames))}.")

print("Face extraction complete.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
No fac

KeyboardInterrupt: 